In [100]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from typing import List

DATA_PATH = '../data/linear_regression_results/linear_regression_hparams_tuning.csv'

In [37]:
hp_df = pd.read_csv(DATA_PATH, header=0, index_col=0)

# Base case where model does not have knowledge on vascular activity
# in previous timestamps
no_back_v_df = hp_df[hp_df['window_len_vascu_back'] == 0]

In [30]:
print(hp_df.columns)

Index(['dataset_name', 'window_len_neuro_back', 'window_len_neuro_forward',
       'window_len_vascu_back', 'poly_degree', 'aggregate_window', 'alpha',
       'test_size', 'mse_train', 'mae_train', 'r2_train', 'mse_test',
       'mae_test', 'r2_test'],
      dtype='object')


In [131]:
def plot_score_per_hp(df: pd.DataFrame = no_back_v_df,
                      metrics: List[str] = ['mse_test', 'mse_train', 'r2_test', 'r2_train'],
                      hyper_param: str = 'window_len_neuro_back',
                      ret_means: bool = False):
    """
    Plot a graph showing the relation between a hyperparameter and a metric.
    Args:
        df: dataframe containing relevant information
        metrics: list of names of columns in df containing the relevant scores
        hyper_param: hyperparameter to plot against (as x-axis)
        ret_means: decide whether should return calculated means

    Returns: (unique values of hyper_param, mean scores for each value)
    """
    vals = df[hyper_param].unique()
    mean_scores = np.zeros((len(vals), len(metrics)), dtype=np.float32)
    for i, val in enumerate(vals):
        relevant_rows = (df[df[hyper_param]  == val])[metrics]
        mean_scores[i] = np.mean(relevant_rows, axis=0)
    mean_scores_df = pd.DataFrame(data=mean_scores, columns=metrics)
    mean_scores_df['vals'] = vals

    fig = go.Figure()
    for i in range(len(metrics)):
        fig.add_trace(
            go.Scatter(x=vals, y=mean_scores[:,i], mode='markers', name=metrics[i])
        )

    fig.update_layout(
    xaxis_title="Value of " + hyper_param,
    yaxis_title="Mean Score of Metric"
    )
    fig.show()
    if ret_means:
        return vals, mean_scores

In [137]:
mse_cols = ['mse_test', 'mse_train']
r2_cols = ['r2_test', 'r2_train']
mae_cols = ['mae_test', 'mae_train']

In [142]:
for hp in ['window_len_neuro_back', 'window_len_neuro_forward']:
    for scores in [mse_cols, r2_cols]:
        plot_score_per_hp(metrics=scores, hyper_param=hp)